In [1]:
import pandas as pd
import datetime

### 데이터 로드

In [2]:
df = pd.read_csv("..\\..\\data\\연도별 재무.csv", encoding = "ms949")

### 매출액은 연결과 개별을 구분지어 필터생성할 것이기 때문에 구분해준다

In [3]:
A = df[(df["Kind"] == "NFS-IFRS(C)") & (df["Item Name "] == "매출액(천원)")]
df.loc[A.index, "Item Name "] = "매출액(연결)"

In [4]:
df.head()

,Symbol,Kind,Item Name,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31,2020-01-31
0,A005930,NFS-IFRS(C),매출액(연결),"206,205,987,000","200,653,482,000","201,866,745,000","239,575,376,000","243,771,415,000",NaN,NaN
1,A005930,NFS-IFRS(S),매출액(천원),"137,825,547,000","135,205,045,000","133,947,204,000","161,915,007,000","170,381,870,000",NaN,NaN
2,A005930,NFS-IFRS(C),영업이익(천원),"25,025,071,000","26,413,442,000","29,240,672,000","53,645,038,000","58,886,669,000",NaN,NaN
3,A005930,NFS-IFRS(C),지배주주순이익(천원),"23,082,499,000","18,694,628,000","22,415,655,000","41,344,569,000","43,890,877,000",NaN,NaN
4,A005930,NFS-IFRS(C),총자본(천원),"168,088,188,000","179,059,805,000","192,963,033,000","214,491,428,000","247,753,177,000",NaN,NaN


### 데이터의 계정이름을 사용하기 쉬운 계정이름으로 변환

In [5]:
계정명 = df["Item Name "].drop_duplicates().tolist()

In [6]:
바꿀계정명 = ["연결매출","매출액", "영업이익", "지배주주순이익", "총자본", "총부채", "자본금", "이익잉여금", 
                 "영업활동으로인한현금흐름", "투자활동으로인한현금흐름", "배당", "주가", "종가", "상장주식수"]

In [7]:
temp = {}

for 계정 in range(len(계정명)):
    temp[계정명[계정]] = 바꿀계정명[계정]
    print(계정명[계정]+ "     ->     " +  바꿀계정명[계정])

매출액(연결)     ->     연결매출
매출액(천원)     ->     매출액
영업이익(천원)     ->     영업이익
지배주주순이익(천원)     ->     지배주주순이익
총자본(천원)     ->     총자본
총부채(천원)     ->     총부채
자본금(천원)     ->     자본금
이익잉여금(천원)     ->     이익잉여금
영업활동으로인한현금흐름(천원)     ->     영업활동으로인한현금흐름
투자활동으로인한현금흐름(천원)     ->     투자활동으로인한현금흐름
수정DPS(보통주,현금)(원)     ->     배당
수정주가(원)     ->     주가
종가(원)     ->     종가
상장주식수(주)     ->     상장주식수


### 바뀐 계정 할당

In [8]:
df = df.replace({"Item Name ": temp})

In [9]:
# 연결매출 판별하는데 사용한 Kind는 이제 필요가 없으므로 drop한다

In [10]:
df = df.drop("Kind", axis=1)

In [11]:
df.head()

,Symbol,Item Name,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31,2020-01-31
0,A005930,연결매출,"206,205,987,000","200,653,482,000","201,866,745,000","239,575,376,000","243,771,415,000",NaN,NaN
1,A005930,매출액,"137,825,547,000","135,205,045,000","133,947,204,000","161,915,007,000","170,381,870,000",NaN,NaN
2,A005930,영업이익,"25,025,071,000","26,413,442,000","29,240,672,000","53,645,038,000","58,886,669,000",NaN,NaN
3,A005930,지배주주순이익,"23,082,499,000","18,694,628,000","22,415,655,000","41,344,569,000","43,890,877,000",NaN,NaN
4,A005930,총자본,"168,088,188,000","179,059,805,000","192,963,033,000","214,491,428,000","247,753,177,000",NaN,NaN


In [12]:
# df 시간변환

In [13]:
df = df.set_index(['Symbol', "Item Name "])
시간변환 = pd.to_datetime(df.columns)
시간변환 = 시간변환.strftime("%Y")
시간변환 = 시간변환.rename("date")
df.columns = 시간변환
df = df.stack().swaplevel().unstack()

In [14]:
df.head()

Item Name                 매출액        배당          상장주식수             연결매출  \
Symbol  date                                                              
A005930 2014  137,825,547,000    400.00    147,299,337  206,205,987,000   
        2015  135,205,045,000    420.00    147,299,337  200,653,482,000   
        2016  133,947,204,000    570.00    140,679,337  201,866,745,000   
        2017  161,915,007,000    850.00    129,098,494  239,575,376,000   
        2018  170,381,870,000  1,416.00  5,969,782,550  243,771,415,000   

Item Name               영업이익    영업활동으로인한현금흐름            이익잉여금          자본금  \
Symbol  date                                                                 
A005930 2014  25,025,071,000  36,975,389,000  169,529,604,000  897,514,000   
        2015  26,413,442,000  40,061,761,000  185,132,014,000  897,514,000   
        2016  29,240,672,000  47,385,644,000  193,086,317,000  897,514,000   
        2017  53,645,038,000  62,162,041,000  215,811,200,000  897,514,000   
        2018  58,886,669,000  67,031,863,000  242,698,956,000  897,514,000   

Item Name            종가      주가         지배주주순이익             총부채  \
Symbol  date                                                      
A005930 2014  1,327,000  26,540  23,082,499,000  62,334,770,000   
        2015  1,260,000  25,200  18,694,628,000  63,119,716,000   
        2016  1,802,000  36,040  22,415,655,000  69,211,291,000   
        2017  2,548,000  50,960  41,344,569,000  87,260,662,000   
        2018     38,700  38,700  43,890,877,000  91,604,067,000   

Item Name                 총자본     투자활동으로인한현금흐름  
Symbol  date                                    
A005930 2014  168,088,188,000  -32,806,408,000  
        2015  179,059,805,000  -27,167,787,000  
        2016  192,963,033,000  -29,658,675,000  
        2017  214,491,428,000  -49,385,216,000  
        2018  247,753,177,000  -52,240,453,000

### 인덱스를 회사 - 시간의 이중 인덱스 변경. 컬럼은 재무 지표들로 변경

### df type 실수로 변환

In [15]:
for 계정 in df.columns:
    df[계정] = df[계정].str.replace(',', "")

df = df.astype(float)

# Making Index

#### 추후에도 lag된 자료들을 쓰고 싶은 계정과목들은 여기서 미리 만든 이후에 lag를 진행한다

In [16]:
df["ROE"] = df["지배주주순이익"] /  df["총자본"]
df["영업이익률"] = df["영업이익"] / df["매출액"]

### #------------------------------------------------------------------------------------------------------------ #

###  lag 진행

In [17]:
lag_1year = df.groupby(["Symbol"]).transform(lambda x : x.shift(1))
lag_2year = df.groupby(["Symbol"]).transform(lambda x : x.shift(2))
lag_3year = df.groupby(["Symbol"]).transform(lambda x : x.shift(3))
lag_4year = df.groupby(["Symbol"]).transform(lambda x : x.shift(4))

In [18]:
def lag_year_df(df, lag_year):
    data= df.copy()
    new_index_list = []
    for index in data.columns:
        new_index = index + " " + str(lag_year) + "Y lag"
        new_index_list.append(new_index)


    data.columns = new_index_list
    
    return data

In [19]:
lag_1year = lag_year_df(lag_1year, 1)
lag_2year = lag_year_df(lag_2year, 2)
lag_3year = lag_year_df(lag_3year, 3)
lag_4year = lag_year_df(lag_4year, 4)

In [20]:
temp = pd.concat([df, lag_1year, lag_2year, lag_3year, lag_4year], axis = 1)

In [21]:
temp.head()

매출액      배당         상장주식수          연결매출          영업이익  \
Symbol  date                                                                   
A005930 2014  1.378255e+11   400.0  1.472993e+08  2.062060e+11  2.502507e+10   
        2015  1.352050e+11   420.0  1.472993e+08  2.006535e+11  2.641344e+10   
        2016  1.339472e+11   570.0  1.406793e+08  2.018667e+11  2.924067e+10   
        2017  1.619150e+11   850.0  1.290985e+08  2.395754e+11  5.364504e+10   
        2018  1.703819e+11  1416.0  5.969783e+09  2.437714e+11  5.888667e+10   

              영업활동으로인한현금흐름         이익잉여금          자본금         종가       주가  \
Symbol  date                                                                
A005930 2014  3.697539e+10  1.695296e+11  897514000.0  1327000.0  26540.0   
        2015  4.006176e+10  1.851320e+11  897514000.0  1260000.0  25200.0   
        2016  4.738564e+10  1.930863e+11  897514000.0  1802000.0  36040.0   
        2017  6.216204e+10  2.158112e+11  897514000.0  2548000.0  50960.0   
        2018  6.703186e+10  2.426990e+11  897514000.0    38700.0  38700.0   

              ...  이익잉여금 4Y lag   자본금 4Y lag  종가 4Y lag  주가 4Y lag  \
Symbol  date  ...                                                    
A005930 2014  ...           NaN          NaN        NaN        NaN   
        2015  ...           NaN          NaN        NaN        NaN   
        2016  ...           NaN          NaN        NaN        NaN   
        2017  ...           NaN          NaN        NaN        NaN   
        2018  ...  1.695296e+11  897514000.0  1327000.0    26540.0   

              지배주주순이익 4Y lag    총부채 4Y lag    총자본 4Y lag  투자활동으로인한현금흐름 4Y lag  \
Symbol  date                                                                    
A005930 2014             NaN           NaN           NaN                  NaN   
        2015             NaN           NaN           NaN                  NaN   
        2016             NaN           NaN           NaN                  NaN   
        2017             NaN           NaN           NaN                  NaN   
        2018    2.308250e+10  6.233477e+10  1.680882e+11        -3.280641e+10   

              ROE 4Y lag  영업이익률 4Y lag  
Symbol  date                            
A005930 2014         NaN           NaN  
        2015         NaN           NaN  
        2016         NaN           NaN  
        2017         NaN           NaN  
        2018    0.137324      0.181571  

[5 rows x 80 columns]

### 만들 필터링 조건들 여기 추가

In [23]:

temp["시가총액"] = temp["상장주식수"] * temp["종가"]

# 포괄손익계산서
# 매출액 단위(천원)
temp["매출액 100억 이상"] = temp["매출액"] > 10000000

#영업이익 3개년 모두 0 이상
temp["영업이익 3년동안 0 이상"] = (temp["영업이익"] > 0) & (temp["영업이익 1Y lag"] > 0) & (temp["영업이익 2Y lag"] > 0)

# 영업이익률 증가
temp["영업이익률 3년동안 증가"] = (temp["영업이익률"] > temp["영업이익률 1Y lag"]) &  (temp["영업이익률"] > temp["영업이익률 2Y lag"]) &  (temp["영업이익률 1Y lag"] > temp["영업이익률 2Y lag"])

# 매출액 3년 동안 증가
temp["매출액 3년동안 증가"] = (temp["매출액"] > temp["매출액 1Y lag"]) &  (temp["매출액"] > temp["매출액 2Y lag"]) &  (temp["매출액 1Y lag"] > temp["매출액 2Y lag"])

# 시가총액 비율
# 이런 식으로 재무데이터와 주가 데이터가 혼합
# 이걸 할 수가 없네

#     temp["순이익시총비율 0.1이상"] = (temp["지배주주순이익"] / temp["시가총액"])  > 0.1
#     temp["영업이익시총비율 0.1이상"] = (temp["영업이익"] / temp["시가총액"]) > 0.1

# 영업이익 1Y lag와 비교하던지


## 재무상태표
# 자본잠식
temp["자본잠식 없음"] = temp["총자본"] > 0

# 부채비율
temp["부채비율"] = temp["총부채"] / temp["총자본"]
temp["부채비율 150% 이하"] = temp["부채비율"] < 1.5

# 자본금
temp["자본금 증가율"] = (temp["자본금"] - temp["자본금 1Y lag"]) / temp["자본금 1Y lag"]
temp["자본금 변동없음"] = temp["자본금 증가율"].abs() < 0.05

# 이익잉여금

temp["이익잉여금이 3년동안 증가"] = (temp["이익잉여금"] > temp["이익잉여금 1Y lag"]) &  (temp["이익잉여금"] > temp["이익잉여금 2Y lag"]) &  (temp["이익잉여금 1Y lag"] > temp["이익잉여금 2Y lag"])


## 현금흐름표

# 영업활동현금흐름
temp["영업활동으로인한현금흐름 판별"] = temp["영업활동으로인한현금흐름"] > 0
temp["3년 중 2년 이상 영업활동으로인한현금흐름 0 초과"] = (temp["영업활동으로인한현금흐름 판별"].rolling(3).sum() > 2)

# 투자활동현금흐름
temp["투자활동으로인한현금흐름 판별"] = temp["투자활동으로인한현금흐름"] < 0
temp["3년 중 2년 이상 투자활동으로인한현금흐름 0 미만"] = (temp["투자활동으로인한현금흐름 판별"].rolling(3).sum() > 2)

## 배당
temp["배당존재"] = temp["배당"] > 0 
temp["배당이 3년동안 증가"] =(temp["배당"] > temp["배당 1Y lag"]) &  (temp["배당"] > temp["배당 2Y lag"]) &  (temp["배당 1Y lag"] > temp["배당 2Y lag"])

## 기타
# ROE가 5% 이상(수익성)
temp["ROE"] = temp["지배주주순이익"] / temp["총자본"]
temp["ROE 5퍼 이상"]  = temp["ROE"] > 0.05

In [24]:
# -순이익 시총비율 0.1이상-과 같이 재무데이터와 주가 데이터를 함께 쓰는 필터의 경우 어떤식으로 처리할 것인지에 대해 결정

In [25]:
필터조건 = ["매출액 100억 이상", "영업이익 3년동안 0 이상", "영업이익률 3년동안 증가", "매출액 3년동안 증가",  
        "자본잠식 없음", "부채비율 150% 이하", "자본금 변동없음", 
        "이익잉여금이 3년동안 증가","3년 중 2년 이상 영업활동으로인한현금흐름 0 초과",
        "3년 중 2년 이상 투자활동으로인한현금흐름 0 미만","배당존재", "배당이 3년동안 증가","ROE 5퍼 이상"]

In [26]:
year_filter = temp[필터조건]

In [27]:
year_filter = year_filter.reset_index().set_index(["date", "Symbol"]).sort_index()

In [28]:
year_filter.to_csv("year_filter.csv", encoding = "ms949")

## 밑에서부터는 추후 프로그램 실행시 데이터를 어떻게 가져올까에 대한 부분들

## 프로그램을 실행시키는 날짜에 따른 reference year를 구한다

In [29]:
def assign_year(date):
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    year = date.year
    month = date.month
    
    if month > 3:
        ref_year = year - 1
        
    else:
        ref_year = year - 2
        
    return str(ref_year)
        

In [30]:
# 예를들어 이 프로그램을 현재 실행시킨다면

In [31]:
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d")

now

'2020-02-06'

In [32]:
get_ref_year = assign_year(now)

In [33]:
get_ref_year

'2018'

In [34]:
year_filter.loc[get_ref_year].head()

,매출액 100억 이상,영업이익 3년동안 0 이상,영업이익률 3년동안 증가,매출액 3년동안 증가,자본잠식 없음,부채비율 150% 이하,자본금 변동없음,이익잉여금이 3년동안 증가,3년 중 2년 이상 영업활동으로인한현금흐름 0 초과,3년 중 2년 이상 투자활동으로인한현금흐름 0 미만,배당존재,배당이 3년동안 증가,ROE 5퍼 이상
Symbol,,,,,,,,,,,,,
A000010,True,True,True,True,True,False,True,True,False,True,False,False,True
A000020,True,True,False,True,True,True,True,True,False,False,True,False,False
A000030,True,True,True,True,True,False,True,True,False,False,True,True,True
A000040,True,False,False,False,True,True,False,False,False,False,False,False,False
A000050,True,True,False,False,True,True,True,True,True,True,True,False,False
